<a href="https://colab.research.google.com/github/gagyeomkim/Deep-Learning-Paper-Review-and-Practice/blob/main/code_practice/Sequence_to_Sequence_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Sequence to Sequence Learning with Neural Networks (NIPS 2014)** 실습
* 본 코드는 기본적으로 **Seq2Seq** 논문의 내용을 따릅니다.
    * 본 논문은 **딥러닝 기반의 자연어 처리** 기법의 기본적인 구성을 이해하고 공부하는 데에 도움을 줍니다.
    * 2020년 기준 가장 뛰어난 번역 모델은 Seq2Seq가 아닌 **Transformer 기반의 모델**입니다.
* 코드 실행 전에 **[런타임]** → **[런타임 유형 변경]** → 유형을 **GPU**로 설정합니다.

#### **데이터 전처리(Preprocessing)**

* **spaCy 라이브러리**: 문장의 토큰화(tokenization), 태깅(tagging) 등의 전처리 기능을 위한 라이브러리
  * 영어(Engilsh)와 독일어(Deutsch) 전처리 모듈 설치

In [ ]:
!pip install torchtext==0.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
%%capture
!python -m spacy download en
!python -m spacy download de

In [ ]:
import spacy
import os

spacy_en = spacy.load('en_core_web_sm') # 영어 토큰화(tokenization)
spacy_de = spacy.load('de_core_news_sm') # 독일어 토큰화(tokenization)

In [ ]:
# # enumerate 사용법
# my_list = ['apple', 'banana', 'cherry']
# for index, value in enumerate(my_list):
#     print(f"인덱스: {index}, 값: {value}")

In [ ]:
# 간단히 토큰화(tokenization) 기능 써보기
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
    print(f"인덱스 {i}: {token.text}")

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: graduate
인덱스 4: student
인덱스 5: .


* 영어(English) 및 독일어(Deutsch) **토큰화 함수** 정의

In [ ]:
# 입력: 독일어(deutsch). 문장을 토큰화한 뒤에 순서를 뒤집는 함수
# 논문에서 입력문장의 경우에는, 입력문장의 순서를 바꾸어서 사용하면 더 좋은 성능을 얻을 수 있다고 다룬다.
def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)][::-1] # start와 stop 생략: 시퀀스의 처음부터 끝까지 모든 요소를 선택
                                                                    # step=-1: 요소를 하나씩 가져오되, 역순으로(오른쪽에서 왼쪽으로)가져옴

# 번역결과: 영어(English) 문장을 토큰화하는 함수
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

* **필드(field)** 라이브러리를 이용해 데이터셋에 대한 구체적인 전처리 내용을 명시합니다.
* 번역 목표
    * 소스(SRC): 독일어
    * 목표(TRG): 영어

In [ ]:
from torchtext.data import Field, BucketIterator
"""
SRC: source의 줄임말로 입력문장
tar: target의 줄임말로 번역하고자 하는 문장
lower=True: 소문자로 처리
"""
SRC = Field(tokenize=tokenize_de, init_token="<sos>", eos_token="<eos>", lower=True)
TRG = Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True)

* 대표적인 영어-독어 번역 데이터셋인 **Multi30k**를 불러옵니다.
    - https://coldjellyfish0227.tistory.com/76

In [ ]:
import os
import gzip
import shutil
import requests

def download_and_unpack(url, dirname, filename):
    response = requests.get(url, stream=True)
    with open(filename, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)

    with gzip.open(filename, 'rb') as f_in:
        with open(os.path.join(dirname, filename[:-3]), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    os.remove(filename)

# Create the necessary directories
if not os.path.exists('.data'):
    os.makedirs('.data')

if not os.path.exists('.data/multi30k'):
    os.makedirs('.data/multi30k')

# URLs for the dataset
urls = [
    "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/train.de.gz",
    "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/train.en.gz",
    "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/val.de.gz",
    "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/val.en.gz",
    "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/test_2016_flickr.de.gz",
    "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/test_2016_flickr.en.gz",
]

# Download and unpack each file
for url in urls:
    filename = url.split("/")[-1]
    download_and_unpack(url, '.data/multi30k', filename)

# Download and unpack the test dataset file
url = "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/test_2016_flickr.en.gz"
filename = url.split("/")[-1]
download_and_unpack(url, '.data/multi30k', filename)

# Rename the file
os.rename('.data/multi30k/test_2016_flickr.en', '.data/multi30k/test2016.en')


# Download and unpack the test dataset file
url = "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/test_2016_flickr.de.gz"
filename = url.split("/")[-1]
download_and_unpack(url, '.data/multi30k', filename)

# Rename the file
os.rename('.data/multi30k/test_2016_flickr.de', '.data/multi30k/test2016.de')

In [ ]:
from torchtext.datasets import Multi30k

train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=(".de", ".en"), fields=(SRC, TRG), root='.data')

In [ ]:
print(f"학습 데이터셋(training dataset) 크기: {len(train_dataset.examples)}개")
print(f"평가 데이터셋(validation dataset) 크기: {len(valid_dataset.examples)}개")
print(f"테스트 데이터셋(testing dataset) 크기: {len(test_dataset.examples)}개")

학습 데이터셋(training dataset) 크기: 29000개
평가 데이터셋(validation dataset) 크기: 1014개
테스트 데이터셋(testing dataset) 크기: 1000개


In [ ]:
# vars 사용법
class SellList:
    snack = 1000
    ice_cream = 700
    coke = 1200
    candy = 1500

print(vars(SellList))

{'__module__': '__main__', 'snack': 1000, 'ice_cream': 700, 'coke': 1200, 'candy': 1500, '__dict__': <attribute '__dict__' of 'SellList' objects>, '__weakref__': <attribute '__weakref__' of 'SellList' objects>, '__doc__': None}


In [ ]:
# 학습 데이터 중 하나를 선택해 출력
print(vars(train_dataset.examples[30])['src'])
print(vars(train_dataset.examples[30])['trg'])

['.', 'steht', 'urinal', 'einem', 'an', 'kaffee', 'tasse', 'einer', 'mit', 'der', ',', 'mann', 'ein']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


* **필드(field)** 객체의 **build_vocab** 메서드를 이용해 영어와 독어의 단어 사전을 생성합니다.
  * **최소 2번 이상** 등장한 단어만을 선택합니다.

In [ ]:
SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset, min_freq=2)

print(f'len(SRC): {len(SRC.vocab)}')
print(f'len(TRG): {len(TRG.vocab)}')

len(SRC): 7853
len(TRG): 5893


In [ ]:
print(TRG.vocab.stoi["abcabc"]) # 없는 단어: 0
print(TRG.vocab.stoi[TRG.pad_token])    # 패딩(padding): 1
print(TRG.vocab.stoi["<sos>"])  # <sos>: 2
print(TRG.vocab.stoi["<eos>"])  # <eos>: 3
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
3
4112
1752


* 한 문장에 포함된 단어가 연속적으로 **LSTM**에 입력되어야 합니다.
    * 따라서 하나의 배치에 포함된 문장들이 가지는 단어의 개수가 유사하도록 만들면 좋습니다.
    * 이를 위해 BucketIterator를 사용합니다.
    * **배치 크기(batch size)**: 128

In [ ]:
import torch

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE =128

# 일반적인 데이터 로더(data loader)의 iterator와 유사하게 사용 가능
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size=BATCH_SIZE,
    device=device
)

In [ ]:
for i, batch in enumerate(train_iterator):
    src = batch.src
    trg = batch.trg

    print(f"첫 번째 배치 크기: {src.shape}")    # (단어의 길이, 배치사이즈)

    # 현재 배치에 있는 하나의 문장에 포함된 정보 출력
    for i in range(src.shape[0]):
        print(f"인덱스 {i}: {src[i][0].item()}")

    # 첫 번째 배치만 확인
    break

첫 번째 배치 크기: torch.Size([27, 128])
인덱스 0: 2
인덱스 1: 4
인덱스 2: 81
인덱스 3: 15
인덱스 4: 7
인덱스 5: 1381
인덱스 6: 967
인덱스 7: 8
인덱스 8: 37
인덱스 9: 10
인덱스 10: 0
인덱스 11: 531
인덱스 12: 11
인덱스 13: 0
인덱스 14: 0
인덱스 15: 6
인덱스 16: 12
인덱스 17: 288
인덱스 18: 97
인덱스 19: 531
인덱스 20: 747
인덱스 21: 7
인덱스 22: 0
인덱스 23: 5
인덱스 24: 3
인덱스 25: 1
인덱스 26: 1


#### **인코더(Encoder) 아키텍처**

* 주어진 소스 문장을 **문맥 벡터(context vector)로 인코딩**합니다.
* LSTM은 hidden state과 cell state을 반환합니다.
    - hidden state: 이전 시점의 출력물이자, 현재 시점의 계산에 직접적으로 사용되는 값
    - cell state: 장기기억을 보관하는 Context Vector
* 하이퍼 파라미터(hyperparameter)
    * **input_dim**: 하나의 단어에 대한 원핫 인코딩 차원(전체 단어의 개수)
    * **embed_dim**: 임베딩(embedding) 차원
    * **hidden_dim**: 히든 상태(hidden state) 차원
    * **n_layers**: RNN 레이어의 개수
    * **dropout_ratio**: 드롭아웃(dropout) 비율

In [ ]:
import torch.nn as nn

# Encoder 아키텍처 정의
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()

        # 임베딩(embedding)은 원-핫 인코딩(one-hot encoding)을 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(input_dim, embed_dim)

        # LSTM 레이어
        self.hidden_dim=hidden_dim
        self.n_layers=n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)

        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더는 소스 문장을 입력으로 받아 Context Vector를 반환
    def forward(self, src):
        # src: (단어개수, 배치크기): 각 단어의 index 정보
        embedded = self.dropout(self.embedding(src))
        # embedded: (단어개수, 배치크기, 임베딩 차원)

        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs: (단어개수, 배치크기, 히든차원): 현재 단어의 출력 정보
        # hidden: (레이어 개수, 배치크기, 히든차원): 현재까지의 모든 단어의 정보
        # cell: (레이어 개수, 배치크기, 히든차원): 현재까지의 모든 단어의 정보

        # 문맥벡터(context vector) 반환
        return hidden, cell

#### **디코더(Decoder) 아키텍처**

* 주어진 문맥 벡터(context vector)를 **타겟 문장으로 디코딩**합니다.
* LSTM은 hidden state과 cell state을 반환합니다.
* 하이퍼 파라미터(hyperparameter)
    * **input_dim**: 하나의 단어에 대한 원핫 인코딩 차원
    * **embed_dim**: 임베딩(embedding) 차원
    * **hidden_dim**: 히든 상태(hidden state) 차원
    * **n_layers**: RNN 레이어의 개수
    * **dropout_ratio**: 드롭아웃(dropout) 비율

In [ ]:
# 디코더(Decoder) 아키텍처 정의
class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()

        # 임베딩(embedding)은 원-핫 인코딩(one-hot encoding)말고 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(output_dim, embed_dim)

        # LSTM 레이어
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)

        # FC 레이어 (인코더와 구조적으로 다른 부분)
        self.output_dim = output_dim
        self.fc_out = nn.Linear(hidden_dim, output_dim)

        # 드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)

    # 디코더는 현재까지 출력된 문장에 대한 정보를 입력으로 받아 타겟 문장을 반환
    def forward(self, input, hidden, cell):
        # input: (배치크기): 단어의 개수는 항상 1개이도록 구현
        # hidden: (레이어개수, 배치크기, 히든차원)
        # cell = context: (레이어개수, 배치크기, 히든차원)
        input = input.unsqueeze(0)
        # input: (단어개수=1, 배치크기)

        embedded = self.dropout(self.embedding(input))
        # embedded: (단어개수, 배치크기, 임베딩 차원)

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output: (단어개수=1, 배치크기, 히든차원): 현재 단어의 출력정보
        # hidden: (레이어개수, 배치크기, 히든차원): 현재까지의 모든 단어의 정보
        # cell: (레이어개수, 배치크기, 히든차원): 현재까지의 모든 단어의 정보

        # 단어 개수는 어차피 1개이므로 0번째 차원 제거
        prediction = self.fc_out(output.squeeze(0))
        # prediction = (배치크기, 출력차원)

        # (현재 출력 단어, 현재까지의 모든 단어의 정보, 현재까지의 모든 단어의 정보)
        return prediction, hidden, cell

#### **Seq2Seq 아키텍처**

* 앞서 정의한 인코더(encoder)와 디코더(decoder)를 가지고 있는 하나의 아키텍처입니다.
    * **인코더(encoder)**: 주어진 소스 문장을 문맥 벡터(context vector)로 인코딩합니다.
    * **디코더(decoder)**: 주어진 문맥 벡터(context vector)를 타겟 문장으로 디코딩합니다.
    * 단, **디코더는 한 단어씩** 넣어서 한 번씩 결과를 구합니다.
* **Teacher forcing**: 디코더의 예측(prediction)을 다음 입력으로 사용하지 않고, 실제 목표 출력(ground-truth)을 다음 입력으로 사용하는 기법

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    # 학습할 때는 완전한 형태의 소스문장, 타겟문장, teacher_forcing_ratio를 넣기
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: (단어개수, 배치크기)
        # trg: (단어개수, 배치크기)
        # 먼저 인코더를 거쳐 문맥 벡터(Context Vector)를 추출
        hidden, cell = self.encoder(src)

        # 디코더(decoder)의 최종 결과를 담을 텐서 객체 만들기
        trg_len = trg.shape[0]  # 단어 개수
        batch_size = trg.shape[1]   # 배치 크기
        trg_vocab_size = self.decoder.output_dim # 출력차원
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # 첫번째 입력은 항상 <sos> 토큰
        input = trg[0, :]

        # 타켓 단어의 개수만큼 반복하여 디코더에 forwarding
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)

            outputs[t] = output #FC를 거쳐서 나온 현재 단어의 출력 정보
            top1 = output.argmax(1) # 가장 확률이 높은 단어의 인덱스 추출

            # teacher_forcing_ratio: 학습할 때 실제 목표 출력(ground-trhuth)을 사용하는 비율
            teacher_force = random.random() < teacher_forcing_ratio
            input = trg[t] if teacher_force else top1   # 현재의 출력 결과를 다음 입력에서 넣기

        return outputs

#### **학습(Training)**

* 하이퍼 파라미터 설정 및 모델 초기화

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENCODER_EMBED_DIM = 256
DECODER_EMBED_DIM = 512
HIDDEN_DIM = 512
N_LAYERS = 2
ENC_DROPOUT_RATIO = 0.5
DEC_DROPOUT_RATIO = 0.5

In [ ]:
# 인코더와 디코더 객체 선언
enc = Encoder(INPUT_DIM, ENCODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, ENC_DROPOUT_RATIO)
dec = Decoder(OUTPUT_DIM, DECODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, DEC_DROPOUT_RATIO)

# Seq2Seq 객체 선언
model = Seq2Seq(enc, dec, device).to(device)

* 논문의 내용대로 $\mathcal{U}(-0.08, 0.08)$의 값으로 **모델 가중치 파라미터 초기화**

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 512)
    (rnn): LSTM(512, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

* 학습 및 평가 함수 정의

In [ ]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
optimizer = optim.Adam(model.parameters())

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [ ]:
# 모델 학습(train) 함수
def train(model, iterator, optimizer, criterion, clip):
    model.train()   # 학습 모드
    epoch_loss = 0

    # 전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)
        # output: (출력 단어 개수, 배치크기, 출력차원)
        output_dim = output.shape[-1]

        # 출력 단어의 인덱스 0은 사용하지 않음
        output = output[1:].view(-1, output_dim)
        # output = ((출력단어의 개수 -1)*batch_size, output_dim)
        trg = trg[1:].view(-1)
        # trg = ((타겟단어의 개수 - 1) * batch size)

        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산

        # 기울기(gradient) clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # 파라미터 업데이트
        optimizer.step()

        # 전체 손실값 계산
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval()    # 평가모드
    epoch_loss = 0

    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            # 평가할 때 teacher forcing은 사용하지 않음
            output = model(src, trg, 0)
            # output: (출력 단어 개수, 배치크기, 출력 차원)
            output_dim = output.shape[-1]

            # 출력 단어의 인덱스 0은 사용하지 않음
            output = output[1:].view(-1, output_dim)
            # output = ((출력 단어의 개수)*배치크기, 출력 차원)
            trg = trg[1:].view(-1)
            #trg = ((타겟 단어의 개수 - 1) * 배치크기)

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실값 계산
            epoch_loss = loss.item()

    return epoch_loss / len(iterator)


* 학습(training) 및 검증(validation) 진행
    * **학습 횟수(epoch)**: 20

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
import time
import math
import random

N_EPOCHS = 20
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()    # 시작시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq.pt')

    print(f"Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s")
    print(f"\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}")
    print(f"\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}")

Epoch: 01 | Time: 0m 51s
	Train Loss: 4.422 | Train PPL: 83.259
	Validation Loss: 0.639 | Validation PPL: 1.895
Epoch: 02 | Time: 0m 43s
	Train Loss: 4.101 | Train PPL: 60.422
	Validation Loss: 0.626 | Validation PPL: 1.870
Epoch: 03 | Time: 0m 50s
	Train Loss: 3.931 | Train PPL: 50.960
	Validation Loss: 0.609 | Validation PPL: 1.838
Epoch: 04 | Time: 0m 50s
	Train Loss: 3.763 | Train PPL: 43.089
	Validation Loss: 0.610 | Validation PPL: 1.841
Epoch: 05 | Time: 0m 47s
	Train Loss: 3.614 | Train PPL: 37.113
	Validation Loss: 0.601 | Validation PPL: 1.823
Epoch: 06 | Time: 0m 48s
	Train Loss: 3.484 | Train PPL: 32.598
	Validation Loss: 0.585 | Validation PPL: 1.795
Epoch: 07 | Time: 0m 47s
	Train Loss: 3.363 | Train PPL: 28.871
	Validation Loss: 0.594 | Validation PPL: 1.811
Epoch: 08 | Time: 0m 49s
	Train Loss: 3.225 | Train PPL: 25.152
	Validation Loss: 0.587 | Validation PPL: 1.798
Epoch: 09 | Time: 0m 50s
	Train Loss: 3.124 | Train PPL: 22.734
	Validation Loss: 0.571 | Validation PPL

In [ ]:
# 학습된 모델 저장
from google.colab import files

files.download('seq2seq.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### **모델 최종 테스트(testing) 결과 확인**

In [ ]:
# !wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/ERgwTMYWR7FMhApROaNvZREBTjEDi00ttSzt8ZNj1PS_5g?download=1 -O seq2seq.pt

In [ ]:
model.load_state_dict(torch.load('seq2seq.pt'))
test_loss = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f}')

Test Loss: 0.552 | Test PPL: 1.738


#### **나만의 데이터로 모델 사용해보기**

In [ ]:
# 번역(translation) 함수
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50):
    model.eval() # 평가 모드

    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # 처음에 <sos> 토큰, 마지막에 <eos> 토큰 붙이기
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    print(f"전체 소스 토큰: {tokens}")

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    print(f"소스 문장 인덱스: {src_indexes}")

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # 인코더(endocer)에 소스 문장을 넣어 문맥 벡터(context vector) 계산
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)

    # 처음에는 <sos> 토큰 하나만 가지고 있도록 하기
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):
        # 이전에 출력한 단어가 현재 단어로 입력될 수 있도록
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token) # 출력 문장에 더하기

        # <eos>를 만나는 순간 끝
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break

    # 각 출력 단어 인덱스를 실제 단어로 변환
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    # 첫 번째 <sos>는 제외하고 출력 문장 반환
    return trg_tokens[1:]

In [ ]:
example_idx = 10

src = vars(test_dataset.examples[example_idx])['src']
trg = vars(test_dataset.examples[example_idx])['trg']

print(f"소스 문장 : {src}")
print(f"타겟 문장 : {trg}")
print("모델 출력 결과:", " ".join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장 : ['.', 'freien', 'im', 'tag', 'schönen', 'einen', 'genießen', 'sohn', 'kleiner', 'ihr', 'und', 'mutter', 'eine']
타겟 문장 : ['a', 'mother', 'and', 'her', 'young', 'song', 'enjoying', 'a', 'beautiful', 'day', 'outside', '.']
전체 소스 토큰: ['<sos>', '.', 'freien', 'im', 'tag', 'schönen', 'einen', 'genießen', 'sohn', 'kleiner', 'ihr', 'und', 'mutter', 'eine', '<eos>']
소스 문장 인덱스: [2, 4, 88, 20, 200, 780, 19, 565, 624, 70, 134, 10, 364, 8, 3]
모델 출력 결과: a mother and her daughter are enjoying a day day . <eos>


In [ ]:
src = tokenize_de("Guten Abend.")

print(f'소스 문장: {src}')
print("모델 출력 결과:", " ".join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장: ['.', 'Abend', 'Guten']
전체 소스 토큰: ['<sos>', '.', 'abend', 'guten', '<eos>']
소스 문장 인덱스: [2, 4, 1163, 3798, 3]
모델 출력 결과: night . <eos>
